In [ ]:
!pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
import os
from datetime import datetime

# Reddit Collation

In [ ]:
# Read Yahoo Finance Dataset
collated_df = pd.read_csv('..\data\meta\Yahoo Finance.csv', index_col = 0)
# Convert Earnings Date and Previous Earnings Date to datetime format
collated_df['Earnings Date'] = pd.to_datetime(collated_df['Earnings Date'], format = '%d/%m/%Y')
collated_df['Previous Earnings Date'] = pd.to_datetime(collated_df['Previous Earnings Date'], format = '%d/%m/%Y')
# Number of days between each quarter
collated_df['Delta'] = (collated_df['Earnings Date'] - collated_df['Previous Earnings Date']).dt.days
sentiment_df = collated_df[['Ticker', 'Earnings Date', 'Previous Earnings Date', 'Delta']].copy(deep = True)
sentiment_lst = sentiment_df.values.tolist()
# Initialise columns for reddit sentiment analysis
collated_df['Reddit_Sentiment'] = np.nan
collated_df['Reddit_Polarity'] = np.nan

In [ ]:
collated_df

,Unnamed: 0.1,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Previous Earnings Date,Price Change,Coefficient Variation of Stock Price,Percentage Change from prev reported EPS,EPS Beats,...,second_strat,third_strat,fourth_strat,first_strat_profit,second_strat_profit,third_strat_profit,fourth_strat_profit,Delta,Reddit_Sentiment,Reddit_Polarity
0,0,2022-09-07,-0.38,-0.35,0.0741,2022-06-01,-0.122655,0.106351,-0.269231,1,...,-0.055921,-0.043755,0.027446,-0.121345,-0.055921,-0.043755,0.027446,98,NaN,NaN
1,1,2022-06-01,-0.36,-0.52,-0.4325,2022-03-17,0.579668,0.219666,-0.217391,0,...,-0.023395,-0.026776,0.074234,0.115225,0.023395,0.026776,-0.074234,76,NaN,NaN
2,2,2022-03-17,0.21,-0.46,-3.2249,2021-12-08,-0.535234,0.180643,-1.600000,0,...,0.098621,0.009671,0.045245,-0.061229,-0.098621,-0.009671,-0.045245,99,NaN,NaN
3,3,2021-12-08,-0.13,-0.35,-1.6977,2021-09-08,-0.160201,0.086445,-0.315789,0,...,-0.067975,-0.023396,-0.124009,-0.039074,0.067975,0.023396,0.124009,91,NaN,NaN
4,4,2021-09-08,-0.17,-0.19,-0.1446,2021-06-09,-0.324167,0.164528,0.545455,0,...,-0.017608,-0.001005,0.000904,0.019482,0.017608,0.001005,-0.000904,91,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,7,2021-02-11,1.46,1.47,0.0096,2020-10-01,0.014390,0.025595,-0.120482,1,...,-0.041046,-0.019826,-0.027814,-0.033166,-0.041046,-0.019826,-0.027814,133,NaN,NaN
975,8,2020-10-01,1.49,1.66,0.1156,2020-07-13,0.023988,0.016441,0.128788,1,...,0.006562,0.015873,-0.003896,0.026538,0.006562,0.015873,-0.003896,80,NaN,NaN
976,9,2020-07-13,1.25,1.32,0.0543,2020-04-28,-0.005354,0.012858,0.168224,1,...,0.021174,0.003347,0.007884,0.016577,0.021174,0.003347,0.007884,76,NaN,NaN
977,10,2020-04-28,1.03,1.07,0.0398,2020-02-13,-0.074019,0.081405,-0.289655,1,...,-0.007443,0.013833,-0.008181,0.014588,-0.007443,0.013833,-0.008181,75,NaN,NaN


In [ ]:
sentiment_lst

In [ ]:
# Change Directory for Twitter Data
os.chdir('../data/datafile/reddit/')

In [ ]:
count = {}
for idx, ticker in enumerate(sentiment_lst):
    try:
        file = f'{ticker[0]}_posts.csv' 
        ticker_df = pd.read_csv(file, index_col = 0)
        if ticker[0] not in count:
            count[ticker[0]] = ticker_df['quarter'].max()
        else:
            count[ticker[0]] -= 1
        ticker_df = ticker_df.loc[ticker_df['quarter'] == count[ticker[0]]].copy(deep = True)
        tmp = np.zeros(ticker[3])
        index = pd.date_range(start=ticker[2], end=ticker[1], freq='D')
        data = pd.Series(tmp, index[1:])
        print(data)
        #create date time column
        ticker_df['date'] = ticker_df['created_utc'].apply(lambda timestamp : datetime.fromtimestamp(timestamp).strftime('%d-%m-%y'))
        ticker_df['date'] = pd.to_datetime(ticker_df['date'], format = '%d-%m-%y')
        ticker_df['collated_score'] = np.nan

        for date in index[:0:-1]:
            mask = ticker_df['date'] == date
            ticker_df.loc[mask,'collated_score'] = ticker_df.loc[mask, 'weighted_score'] * (ticker_df.loc[mask,'score']/ticker_df.loc[mask,'score'].sum())
            data[date] = ticker_df.loc[mask, 'collated_score'].sum()

        data.interpolate(inplace=True)
        collated_df.loc[(collated_df['Earnings Date'] == ticker[1]) & (collated_df['Previous Earnings Date'] == ticker[2]) & (collated_df['Ticker'] == ticker[0]), 'Reddit_Sentiment'] = data.mean()
    except:
        pass

In [ ]:
collated_df['Reddit_Sentiment'].isna().value_counts()

False    965
True      14
Name: Reddit_Sentiment, dtype: int64

In [ ]:
collated_df

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Previous Earnings Date,Price Change,Coefficient Variation of Stock Price,Percentage Change from prev reported EPS,EPS Beats,Ticker,Sector,Delta,Reddit_Sentiment,Reddit_Polarity
0,2022-09-07,-0.38,-0.35,0.0741,2022-06-01,-0.122655,0.106351,-0.269231,1,GME,Consumer Cyclical,98,-0.025041,NaN
1,2022-06-01,-0.36,-0.52,-0.4325,2022-03-17,0.579668,0.219666,-0.217391,0,GME,Consumer Cyclical,76,-0.169463,NaN
2,2022-03-17,0.21,-0.46,-3.2249,2021-12-08,-0.535234,0.180643,-1.600000,0,GME,Consumer Cyclical,99,-0.180444,NaN
3,2021-12-08,-0.13,-0.35,-1.6977,2021-09-08,-0.160201,0.086445,-0.315789,0,GME,Consumer Cyclical,91,0.391805,NaN
4,2021-09-08,-0.17,-0.19,-0.1446,2021-06-09,-0.324167,0.164528,0.545455,0,GME,Consumer Cyclical,91,-0.136852,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2021-02-11,1.46,1.47,0.0096,2020-10-01,0.014390,0.025595,-0.120482,1,PEP,Consumer Defensive,133,0.000650,NaN
8,2020-10-01,1.49,1.66,0.1156,2020-07-13,0.023988,0.016441,0.128788,1,PEP,Consumer Defensive,80,0.008174,NaN
9,2020-07-13,1.25,1.32,0.0543,2020-04-28,-0.005354,0.012858,0.168224,1,PEP,Consumer Defensive,76,0.157317,NaN
10,2020-04-28,1.03,1.07,0.0398,2020-02-13,-0.074019,0.081405,-0.289655,1,PEP,Consumer Defensive,75,0.019263,NaN


In [ ]:
collated_df

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Previous Earnings Date,Price Change,Coefficient Variation of Stock Price,Percentage Change from prev reported EPS,EPS Beats,Ticker,Sector,Delta,Reddit_Sentiment,Reddit_Polarity
0,2022-09-07,-0.38,-0.35,0.0741,2022-06-01,-0.122655,0.106351,-0.269231,1,GME,Consumer Cyclical,98,0.129122,NaN
1,2022-06-01,-0.36,-0.52,-0.4325,2022-03-17,0.579668,0.219666,-0.217391,0,GME,Consumer Cyclical,76,0.172532,NaN
2,2022-03-17,0.21,-0.46,-3.2249,2021-12-08,-0.535234,0.180643,-1.600000,0,GME,Consumer Cyclical,99,0.207400,NaN
3,2021-12-08,-0.13,-0.35,-1.6977,2021-09-08,-0.160201,0.086445,-0.315789,0,GME,Consumer Cyclical,91,0.286349,NaN
4,2021-09-08,-0.17,-0.19,-0.1446,2021-06-09,-0.324167,0.164528,0.545455,0,GME,Consumer Cyclical,91,0.240946,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2021-02-11,1.46,1.47,0.0096,2020-10-01,0.014390,0.025595,-0.120482,1,PEP,Consumer Defensive,133,0.076131,NaN
8,2020-10-01,1.49,1.66,0.1156,2020-07-13,0.023988,0.016441,0.128788,1,PEP,Consumer Defensive,80,0.064342,NaN
9,2020-07-13,1.25,1.32,0.0543,2020-04-28,-0.005354,0.012858,0.168224,1,PEP,Consumer Defensive,76,0.088555,NaN
10,2020-04-28,1.03,1.07,0.0398,2020-02-13,-0.074019,0.081405,-0.289655,1,PEP,Consumer Defensive,75,0.063063,NaN


# Get Distribution for Reddit Posts

In [ ]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1w6Fl0Jpxs6henghVWCdFh9UGHBdzVRjf/BT4222 Project/Final Paper/Codes/Datasets/Reddit Data'

In [ ]:
df = pd.read_csv('../data/alternatives_alpha_datasets/Reddit_Sentiments_equalweight.csv')

In [ ]:
df.tail(5)

,Unnamed: 0,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Previous Earnings Date,Price Change,Coefficient Variation of Stock Price,Percentage Change from prev reported EPS,EPS Beats,Ticker,Sector,Delta,Reddit_Sentiment,Reddit_Polarity
974,7,2021-02-11,1.46,1.47,0.0096,2020-10-01,0.014390,0.025595,-0.120482,1,PEP,Consumer Defensive,133,0.028897,0.352831
975,8,2020-10-01,1.49,1.66,0.1156,2020-07-13,0.023988,0.016441,0.128788,1,PEP,Consumer Defensive,80,0.046342,0.258866
976,9,2020-07-13,1.25,1.32,0.0543,2020-04-28,-0.005354,0.012858,0.168224,1,PEP,Consumer Defensive,76,0.117191,0.320746
977,10,2020-04-28,1.03,1.07,0.0398,2020-02-13,-0.074019,0.081405,-0.289655,1,PEP,Consumer Defensive,75,0.061396,0.416460
978,11,2020-02-13,1.44,1.45,0.0097,2019-10-03,0.098277,0.025865,-0.076923,1,PEP,Consumer Defensive,133,0.098184,0.401170


In [ ]:
count = {}
for idx, ticker in enumerate(sentiment_lst):
    try:
        file = f'{ticker[0]}_posts.csv'
        ticker_df = pd.read_csv(file, index_col = 0)
        if ticker[0] not in count:
            count[ticker[0]] = ticker_df['quarter'].max()
        else:
            count[ticker[0]] -= 1
        ticker_df = ticker_df.loc[ticker_df['quarter'] == count[ticker[0]]].copy(deep = True)
        polarity_dist = ticker_df[['quarter','weighted_score']].groupby(by = ['quarter']).std()
        value = list(polarity_dist['weighted_score'])[0]
        print(value)
        collated_df.loc[(collated_df['Earnings Date'] == ticker[1]) & (collated_df['Previous Earnings Date'] == ticker[2]) & (collated_df['Ticker'] == ticker[0]), 'Reddit_Polarity'] = value
    except:
        pass

In [ ]:
collated_df = collated_df.drop(columns = ['Reddit_Polarity_Dist'])

bold text# Reddit Data Imputation

In [ ]:
# Fill missing values with 0
collated_df['Reddit_Polarity'] = collated_df['Reddit_Polarity'].fillna(0)

In [ ]:
# collated_df.to_csv('Reddit_Sentiments_equalweights.csv')